In [1]:
from bods_client.models.base import APIError
from bods_client.client import BODSClient
from bods_client.models import BoundingBox, Siri, SIRIVMParams

import csv
import time, datetime

In [2]:
API_KEY = ""

client = BODSClient(api_key=API_KEY)

def query_vehicle_data(client, params):
    siri_response = client.get_siri_vm_data_feed(params=params)
    if type(siri_response) == APIError:
        raise APIError(siri_response)
    siri = Siri.from_bytes(siri_response)
    v_data = siri.service_delivery.vehicle_monitoring_delivery.vehicle_activities
    
    return v_data

def create_csv_dict(v_data):
    timestamp = v_data.recorded_at_time
    j_data = v_data.monitored_vehicle_journey.model_dump()
    j_data['recorded_at_time'] = timestamp

    return j_data


In [3]:
bounding_box = BoundingBox(
    **{
        "min_latitude": 50.72,
        "max_latitude": 50.772,
        "min_longitude": -3.51,
        "max_longitude": -3.499,
    }
)

params = SIRIVMParams(bounding_box=bounding_box)

csv_file = 'buses.csv'



In [4]:
v_data = []
# get some initial bus data
while(len(v_data) == 0):
    v_data = query_vehicle_data(client, params)
    now = datetime.datetime.now()
    now = now.strftime("%Y-%m-%d %H:%M:%S")
    if len(v_data) > 0:
        print(f'{now} found {len(v_data)} buses!')
    else:
        print(f'{now} - no buses found.')
        time.sleep(30)

# Open the CSV file for writing and write the header
with open(csv_file, "w", newline="") as csvfile:
    j_data = create_csv_dict(v_data[0])
    writer = csv.DictWriter(csvfile, fieldnames=j_data.keys())
    writer.writeheader()

    # write first set of data
    for v in v_data:
        j_row = create_csv_dict(v)
        print(f'Writing {j_row}')
        writer.writerow(j_row)

    # now read more bus data
    while True:
        time.sleep(30)

        v_data = query_vehicle_data(client, params)
        now = datetime.datetime.now()
        now = now.strftime("%Y-%m-%d %H:%M:%S")
        if len(v_data) > 0:
            print(f'{now} - Found {len(v_data)} buses!')
            for v in v_data:
                j_row = create_csv_dict(v)
                print(f'Writing: {j_row}')
                writer.writerow(j_row)
        else:
            print(f'{now} - No buses found.')

2024-10-20 20:15:12 - no buses found.
2024-10-20 20:15:42 - no buses found.
2024-10-20 20:16:12 - no buses found.
2024-10-20 20:16:43 found 1 buses!
Writing {'bearing': 78.0, 'block_ref': None, 'framed_vehicle_journey_ref': {'data_frame_ref': datetime.date(2024, 10, 20), 'dated_vehicle_journey_ref': '422'}, 'vehicle_journey_ref': None, 'destination_name': 'Tesco Filling Stati', 'destination_ref': '1100DEC10345', 'origin_name': 'Tesco Filling Stati', 'origin_ref': '1100DEC10345', 'origin_aimed_departure_time': datetime.datetime(2024, 10, 20, 17, 30, tzinfo=TzInfo(UTC)), 'direction_ref': 'inbound', 'published_line_name': 'J', 'line_ref': 'J', 'vehicle_location': {'longitude': -3.508751, 'latitude': 50.730759}, 'operator_ref': 'SDVN', 'vehicle_ref': 'SDVN-36230', 'recorded_at_time': datetime.datetime(2024, 10, 20, 19, 16, 32, tzinfo=TzInfo(UTC))}
2024-10-20 20:17:13 - Found 1 buses!
Writing: {'bearing': 78.0, 'block_ref': None, 'framed_vehicle_journey_ref': {'data_frame_ref': datetime.dat

KeyboardInterrupt: 

In [30]:
query_vehicle_data(client, params)

[VehicleActivity(recorded_at_time=datetime.datetime(2024, 10, 20, 17, 47, 5, tzinfo=TzInfo(UTC)), item_identifier='SCBODS:VEHICLESTATUSRT:8195:9330', valid_until_time=datetime.datetime(2024, 10, 20, 17, 52, 17, 317000, tzinfo=TzInfo(UTC)), monitored_vehicle_journey=MonitoredVehicleJourney(bearing=96.0, block_ref=None, framed_vehicle_journey_ref=FramedVehicleJourneyRef(data_frame_ref=datetime.date(2024, 10, 20), dated_vehicle_journey_ref='449'), vehicle_journey_ref=None, destination_name='Court Royal', destination_ref='1100DEA57267', origin_name='St Davids Station', origin_ref='1100DEZ05173', origin_aimed_departure_time=datetime.datetime(2024, 10, 20, 17, 30, tzinfo=TzInfo(UTC)), direction_ref='outbound', published_line_name='4', line_ref='4', vehicle_location=VehicleLocation(longitude=-3.505854, latitude=50.721523), operator_ref='SDVN', vehicle_ref='SDVN-24148'))]